# 如何在聊天模型中使用少量示例
:::info 前提条件
本指南假定您已熟悉以下概念：- [提示模板](/docs/concepts/prompt_templates)- [示例选择器](/docs/concepts/example_selectors)- [聊天模型](/docs/concepts/chat_models)- [向量数据库](/docs/concepts/vectorstores)
:::
本指南介绍了如何通过示例输入和输出来引导聊天模型。为模型提供少量此类示例的方法称为[少样本提示](/docs/concepts/few_shot_prompting/)，这是一种简单而有效的方式，既能指导生成过程，又能在某些情况下显著提升模型表现。
关于如何最好地进行少样本提示（few-shot prompting），目前似乎尚未形成坚实的共识，且最优的提示编译方式可能因模型而异。因此，我们提供了诸如 [FewShotChatMessagePromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.few_shot.FewShotChatMessagePromptTemplate.html?highlight=fewshot#langchain_core.prompts.few_shot.FewShotChatMessagePromptTemplate) 这样的少样本提示模板作为灵活的起点，您可以根据需要对其进行修改或替换。
小样本提示模板的目标是根据输入动态选择示例，然后将这些示例格式化到最终提示中供模型使用。
**注意：** 以下代码示例仅适用于聊天模型，因为`FewShotChatMessagePromptTemplates`的设计目的是输出格式化的[聊天消息](/docs/concepts/messages)，而非纯字符串。如需查看适用于纯字符串模板（兼容补全模型LLM）的类似小样本提示示例，请参阅[小样本提示模板](/docs/how_to/few_shot_examples/)指南。

## 固定示例
最基本的（也是最常见的）少量示例提示技术是使用固定的提示示例。这样你可以选择一个链，对其进行评估，并避免在生产中担心额外的变动部分。
模板的基本组成部分包括：- `examples`: 包含在最终提示中的字典示例列表。- `example_prompt`: 通过其 [`format_messages`](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html?highlight=format_messages#langchain_core.prompts.chat.ChatPromptTemplate.format_messages) 方法将每个示例转换为一条或多条消息。常见的做法是将每个示例转换为一条人类消息和一条AI消息回复，或者一条人类消息后跟一条函数调用消息。
以下是一个简单的演示。首先，定义你想要包含的示例。让我们给这个大型语言模型一个不熟悉的数学运算符，用“🦜”表情符号表示：

In [1]:
%pip install -qU langchain langchain-openai langchain-chroma

import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass()

如果我们尝试询问模型这个表达式的结果是什么，它将会失败：

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

model.invoke("What is 2 🦜 9?")

AIMessage(content='The expression "2 🦜 9" is not a standard mathematical operation or equation. It appears to be a combination of the number 2 and the parrot emoji 🦜 followed by the number 9. It does not have a specific mathematical meaning.', response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 17, 'total_tokens': 71}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-aad12dda-5c47-4a1e-9949-6fe94e03242a-0', usage_metadata={'input_tokens': 17, 'output_tokens': 54, 'total_tokens': 71})

现在让我们看看如果给大语言模型提供一些示例会发生什么。我们将在下面定义几个例子：

In [5]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

examples = [
    {"input": "2 🦜 2", "output": "4"},
    {"input": "2 🦜 3", "output": "5"},
]

接下来，将它们整合到少样本提示模板中。

In [6]:
# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.invoke({}).to_messages())

[HumanMessage(content='2 🦜 2'), AIMessage(content='4'), HumanMessage(content='2 🦜 3'), AIMessage(content='5')]


最后，我们按照如下方式组装最终提示，直接将`few_shot_prompt`传入`from_messages`工厂方法，并与模型一起使用：

In [7]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

现在让我们向模型提出初始问题，看看它的表现如何：

In [8]:
from langchain_openai import ChatOpenAI

chain = final_prompt | model

chain.invoke({"input": "What is 2 🦜 9?"})

AIMessage(content='11', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 60, 'total_tokens': 61}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5ec4e051-262f-408e-ad00-3f2ebeb561c3-0', usage_metadata={'input_tokens': 60, 'output_tokens': 1, 'total_tokens': 61})

我们可以看到，模型现在已经从给定的少量示例中推断出鹦鹉表情符号代表加法！
## 动态少样本提示
有时你可能希望根据输入从整体集合中仅选择少量示例进行展示。为此，你可以将传入`FewShotChatMessagePromptTemplate`的`examples`替换为`example_selector`。其余组件与上述保持一致！我们的动态少量示例提示模板将如下所示：
- `example_selector`: 负责为给定输入选择少样本示例（以及返回顺序）。这些选择器实现了 [BaseExampleSelector](https://python.langchain.com/api_reference/core/example_selectors/langchain_core.example_selectors.base.BaseExampleSelector.html?highlight=baseexampleselector#langchain_core.example_selectors.base.BaseExampleSelector) 接口。一个常见示例是基于向量存储的 [SemanticSimilarityExampleSelector](https://python.langchain.com/api_reference/core/example_selectors/langchain_core.example_selectors.semantic_similarity.SemanticSimilarityExampleSelector.html?highlight=semanticsimilarityexampleselector#langchain_core.example_selectors.semantic_similarity.SemanticSimilarityExampleSelector)- `example_prompt`: 通过其 [`format_messages`](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html?highlight=chatprompttemplate#langchain_core.prompts.chat.ChatPromptTemplate.format_messages) 方法将每个示例转换为1条或多条消息。常见的做法是将每个示例转换为一条人类消息和一条AI消息回复，或者一条人类消息后跟一条函数调用消息。
这些可以再次与其他消息和聊天模板组合，以构建您的最终提示。
让我们通过一个示例来了解 `SemanticSimilarityExampleSelector` 的工作原理。由于该实现使用向量存储（vectorstore）基于语义相似性来选择示例，我们首先需要填充该存储。这里的核心思想是，我们希望搜索并返回与文本输入最相似的示例，因此我们会对提示示例的 `values` 进行嵌入（embedding），而不是考虑键（keys）：

In [9]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

examples = [
    {"input": "2 🦜 2", "output": "4"},
    {"input": "2 🦜 3", "output": "5"},
    {"input": "2 🦜 4", "output": "6"},
    {"input": "What did the cow say to the moon?", "output": "nothing at all"},
    {
        "input": "Write me a poem about the moon",
        "output": "One for the moon, and one for me, who are we to talk about the moon?",
    },
]

to_vectorize = [" ".join(example.values()) for example in examples]
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

### 创建 `example_selector`
创建好向量存储后，我们就可以构建 `example_selector` 了。这里我们将单独调用它，并设置 `k` 参数值，使其仅获取与输入最接近的两个示例。

In [10]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

# The prompt template will load examples by passing the input do the `select_examples` method
example_selector.select_examples({"input": "horse"})

[{'input': 'What did the cow say to the moon?', 'output': 'nothing at all'},
 {'input': '2 🦜 4', 'output': '6'}]

### 创建提示模板
我们现在组装提示模板，使用上面创建的 `example_selector`。

In [11]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# Define the few-shot prompt.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    # The input variables select the values to pass to the example_selector
    input_variables=["input"],
    example_selector=example_selector,
    # Define how each example will be formatted.
    # In this case, each example will become 2 messages:
    # 1 human, and 1 AI
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

print(few_shot_prompt.invoke(input="What's 3 🦜 3?").to_messages())

[HumanMessage(content='2 🦜 3'), AIMessage(content='5'), HumanMessage(content='2 🦜 4'), AIMessage(content='6')]


我们可以将这个少量示例的聊天消息提示模板传入另一个聊天提示模板中：

In [12]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

print(few_shot_prompt.invoke(input="What's 3 🦜 3?"))

messages=[HumanMessage(content='2 🦜 3'), AIMessage(content='5'), HumanMessage(content='2 🦜 4'), AIMessage(content='6')]


### 与聊天模型一起使用
最后，你可以将你的模型连接到少样本提示上。

In [13]:
chain = final_prompt | ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

chain.invoke({"input": "What's 3 🦜 3?"})

AIMessage(content='6', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 60, 'total_tokens': 61}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d1863e5e-17cd-4e9d-bf7a-b9f118747a65-0', usage_metadata={'input_tokens': 60, 'output_tokens': 1, 'total_tokens': 61})

## 后续步骤
你现在已经学会了如何在聊天提示中添加少量示例。
接下来，请查阅本节中关于提示模板的其他操作指南，相关的[文本补全模型少样本学习操作指南](/docs/how_to/few_shot_examples)，或其他[示例选择器操作指南](/docs/how_to/example_selectors/)。